In [1]:
# ALO가 있는 곳을 import 하기 위해 dir 변경
import os

#### ALO 선언하기 및 TCR setup 하기 

📜 TCR : http://mod.lge.com/hub/dxadvtech/aicontents/tcr.git   
📜 GCR : http://mod.lge.com/hub/dxadvtech/aicontents/gcr.git    
📜 Forecast: http://mod.lge.com/hub/dxadvtech/aicontents/biz_forecasting.git     


In [4]:
%pip install -r requirements.txt

In [2]:
from src.alo import ALO

alo = ALO()

alolib git pull success.
패키지 설치 성공



In [ ]:
alo.init("http://mod.lge.com/hub/dxadvtech/aicontents/tcr.git")
alo.set_asset_structure()

In [ ]:
# asset이 동작하는 argument를 alo동작에 필요한 structure의 args라는 변수에 입력
# experimental_plan의 user_pamameter의 train_pipeline의 0번 스텝(input) 에서 args를 가지고 와서 alo.assset_structure.args에 저장
alo.asset_structure.args = alo.get_args("train_pipeline", 0) # pipleline name, pipeline_step(0, 1, 2, 3)

In [ ]:
# alo asset의 설치하기 위한 정보를 asset_config라는 변수에 담는다
asset_config = alo.asset_source['train_pipeline']
asset_config

In [ ]:
# train pipeline에서 externel data path에 입력되어 있는 데이터를 가지고 온다
alo.external_load_data('train_pipeline')

In [ ]:
# train pipeline의 asset들을 설치 한다(once : 이미 설치되어 있으면 설치 하지 않는다, every : 설치가 되어 있어도 다시 설치 한다)
# experimental_plan.yaml의 control의 get_asset_source 키의 값을 사용한다
alo.install_steps('train_pipeline', 'once')

In [ ]:
# 입력 받은 asset을 동작시킨다. 첫 번째 step (0번 step) 을 구동
alo.asset_structure = alo.process_asset_step(asset_config[0], 0, "train_pipeline", alo.asset_structure)

In [ ]:
# 1번 step의 args를 받아야 한다
alo.asset_structure.args = alo.get_args("train_pipeline", 1)

In [ ]:
# 1번 step의 asset을 실행한다
alo.process_asset_step(asset_config[1], 1, "train_pipeline", alo.asset_structure)